# Get All EC2 Instance

In [ ]:
import os, sys
from datetime import datetime
import pandas as pd
import boto3
from IPython.display import display
from botocore.config import Config

# 0. Core Function

### Credential Part

In [ ]:
def createAWSCLISession(aws_access_key_id="", aws_secret_access_key="", aws_session_token="", credential_profile=""):
    if aws_access_key_id != "" and aws_secret_access_key != "":
        # Can Specific Key Here
        session = boto3.Session(
            aws_access_key_id=aws_access_key_id,
            aws_secret_access_key=aws_secret_access_key,
            aws_session_token=aws_session_token
        )

    if credential_profile != "":
        session = boto3.Session(profile_name=credential_profile)
    
    return session

### EC2 Instances

In [ ]:
def extractEC2InstanceName(tags=[]):
    for tag in tags:
        if tag["Key"]=="Name":
            return tag['Value']

def extractOSFromImage(ImageId=""):
    images_info = EC2.describe_images(ImageIds=[ImageId])
    if len(images_info["Images"]) <= 0: # and  in images_info["Images"]:
        return "N/A - Not Windows"
    else:
        return images_info["Images"][0]["PlatformDetails"]

def extractEC2Fields(ReservationId="", info={}):    
    return {
        "reservationId": ReservationId,
        "instanceId":info["InstanceId"],
        "PrivateIpAddress":info["PrivateIpAddress"] if "PrivateIpAddress" in info else "",
        "Platform":info["Platform"] if "Platform" in info else extractOSFromImage(info["ImageId"]),
        "instanceName":extractEC2InstanceName(tags=info["Tags"]),
        "tags":info["Tags"]
    }  

In [ ]:
credential_profiles = [
]

In [ ]:
region_name = 'ap-southeast-1'
EC2config = Config(
    region_name = region_name
)

In [ ]:
for credential_profile in credential_profiles:
    print("\nCredential Profile : " + credential_profile)
    
    session = createAWSCLISession(credential_profile=credential_profile)
    EC2 = session.client('ec2', config=EC2config)
    
    ec2_reserved = EC2.describe_instances()["Reservations"]
    
    ec2_assets = []
    for reserve in ec2_reserved:
        ec2_assets += [extractEC2Fields(ReservationId=reserve["ReservationId"], info=info) for info in reserve["Instances"]]
    
    ec2_assets_df = pd.DataFrame(ec2_assets)
    ec2_assets_mergeTag_df = pd.concat([ec2_assets_df, ec2_assets_df["tags"].apply(pd.Series)], axis=1)
    ec2_assets_mergeTag_df = ec2_assets_mergeTag_df.drop(columns="tags")
    
    filepath="outputs/" + credential_profile
    ec2_assets_mergeTag_df.to_excel(filepath  + ".xlsx", sheet_name='sheet', index=False, encoding='utf-8')